# 사용자 정의 Evaluator로평가

사용자 정의 LLM평가자를 구성하거나 Heuristic평가자를 구성할 수 있다.

LLM이 답변이 좋은지 나쁜지 판단하는데, 룰 기반의 평가자를 만들수도 있다.

어떻게보면 이게 제일 유용할 수 있다. 

prompting방식으로 평가자를 만드는 것이다.

**RUN은 LLM의 답변이 담기는 매개변수, Example은 정답 답변이다.**

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging
logging.langsmith("CH16-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


In [3]:
from myrag import PDFRAG
from langchain_openai import ChatOpenAI

rag = PDFRAG(
    "data/SPRI_AI_Brief_2023년12월호_F.pdf",
    ChatOpenAI(model="gpt-4o-mini")
)

retriever = rag.create_retriever()

chain = rag.create_chain(retriever)

chain.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은?")

"삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스'입니다."

In [4]:
def ask_question(inputs:dict):
    return {"answer": chain.invoke(inputs["question"])}

### 사용자 정의 Evaluator 구성
아래의 사용자 정의 함수의 입력 매개변수와 반환 값 형식을 지켜서 생성할 수 있습니다.

**사용자 정의 함수**

- 입력으로는 `Run` 과 `Example` 을 받고 출력으로는 `dict` 를 반환합니다.
- 반환 값은 `{"key": "score_name", "score": score}` 형식으로 구성됩니다.
아래는 간단한 예시 함수를 정의하였습니다. 답변에 상관없이 1~10 사이의 랜덤 점수를 반환합니다.

In [5]:
from langsmith.schemas import Run, Example
import random

def random_score_evaluator(run: Run, example: Example) -> dict:
    score = random.randint(0,10)
    return{"key": "randon_score", "score": score}

In [6]:
from langsmith.evaluation import evaluate

dataset_name = "RAG_EVAL_DATASET_teddynote"

experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[random_score_evaluator],
    experiment_prefix="CUSTOM-EVAL",
    metadata={
        "variant": "랜덤 점수 평가자"
    }
)

View the evaluation results for experiment: 'CUSTOM-EVAL-a8a44c5e' at:
https://smith.langchain.com/o/2d0ce887-3f7f-59af-8d5e-12c1371ef5d5/datasets/334d943e-3086-4d03-91ae-4b5145224ffc/compare?selectedSessions=3bcb15ef-c0b9-46b0-890c-70f02851ae2b




0it [00:00, ?it/s]

Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'


### Custom LLM-as-Judge
이번에는 LLM Chain 을 만들어서 평가자로 활용하겠습니다.

먼저, `context`, `answer`, `question` 을 반환하는 함수를 정의합니다.

In [7]:
def context_answer_rag_answer(inputs:dict):
    context = retriever .invoke(inputs["question"])
    return {
        "context": "\n".join([doc.page_content for doc in context]),
        "answer": chain.invoke(inputs["question"]),
        "question": inputs["question"]
    }

In [9]:
from langchain import hub

llm_evaluator_prompt = hub.pull("teddynote/context-answer-evaluator")
llm_evaluator_prompt.pretty_print()


As an LLM evaluator (judge), please assess the LLM's response to the given question. Evaluate the response's accuracy, comprehensiveness, and context precision based on the provided context. After your evaluation, return only the numerical scores in the following format:
Accuracy: [score]
Comprehensiveness: [score]
Context Precision: [score]
Final: [normalized score]
Grading rubric:

Accuracy (0-10 points):
Evaluate how well the answer aligns with the information provided in the given context.

0 points: The answer is completely inaccurate or contradicts the provided context
4 points: The answer partially aligns with the context but contains significant inaccuracies
7 points: The answer mostly aligns with the context but has minor inaccuracies or omissions
10 points: The answer fully aligns with the provided context and is completely accurate


Comprehensiveness (0-10 points):

0 points: The answer is completely inadequate or irrelevant
3 points: The answer is accurate but too brief t

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

custom_llm_evaluator = (
    llm_evaluator_prompt
    | ChatOpenAI(temperature=0, model="gpt-4o-mini")
    | StrOutputParser()
)

In [13]:
output = context_answer_rag_answer(
    {"question": "삼성전자가 자체 개발한 생성형 AI의 이름은?"}
)

In [14]:
print(output)

{'context': '▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··························································· 10\n   ▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································ 11\n   ▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파 전망··········································· 12\nSPRi AI Brief |  \n2023-12월호\n10\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\nKEY Contents\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, \n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유\n어시스턴트를 적용한 구글 픽셀(Pixel)과 경쟁할 것으로 예상\n☞ 출처 : 삼성전자, ‘삼성 AI 포럼’서 자체 개발 생성형 AI ‘삼성 가우스’ 공개, 2023.11.08.\n삼성전

`custom_evaluator` 함수를 정의합니다.

- `run.outputs`: RAG 체인이 생성한 answer, context, question 을 가져옵니다.
- `example.outputs`: 데이터셋의 정답 답변을 가져옵니다.

In [16]:
from langsmith.schemas import Run, Example

def custom_evaluator(run: Run, example: Example) -> dict:
    llm_answer = run.outputs.get("answer", "")
    context = run.outputs.get("context", "")
    question = example.outputs.get("question", "")

    # 랜덤 점수 반환
    score = custom_llm_evaluator.invoke(
        {"question": question, "answer": llm_answer, "context": context}
    )
    return {"key": "custom_score", "score": float(score)}

In [17]:
from langsmith.evaluation import evaluate

dataset_name = "RAG_EVAL_DATASET_teddynote"

experiment_results = evaluate(
    context_answer_rag_answer,
    data=dataset_name,
    evaluators=[custom_evaluator],
    experiment_prefix="CUSTOM-LMM-EVAL",
    metadata={
        "variant": "Custom LLM Evaluator 활용한 평가"
    }
)

View the evaluation results for experiment: 'CUSTOM-LMM-EVAL-ea406cb2' at:
https://smith.langchain.com/o/2d0ce887-3f7f-59af-8d5e-12c1371ef5d5/datasets/334d943e-3086-4d03-91ae-4b5145224ffc/compare?selectedSessions=8c023e7a-744b-4d7a-842c-42708c84d1c1




0it [00:00, ?it/s]

Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
Error running target function: 'question'
